In [0]:
%sql
create catalog maven_catalog;

---------------------------------------------------------------------------
UnknownException                          Traceback (most recent call last)
File <command-6859696035149934>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'use catalog maven_catalog;\ncreate schema maven;\nuse schema maven;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:192, in SqlMagic.sql(self, line, cell)
    186 except BaseException as e:
    187   

In [0]:
%sql
use catalog maven_catalog;


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:498)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:756)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:512)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:632)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$5(UsageLogging.scala:659)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:146)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Att

In [0]:
%sql
create schema if not exists data_set

In [0]:
raw_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://mavenmarket@sgmavenmarkets.dfs.core.windows.net/raw/stores/MavenMarket_Stores.csv")

delta_table = "products_delta_table"
raw_df.write.format("delta").mode("overwrite").saveAsTable(delta_table)
display(spark.read.table(delta_table))

---------------------------------------------------------------------------
UnknownException                          Traceback (most recent call last)
File <command-7867010594497504>, line 7
      1 raw_df = spark.read.format("csv") \
      2     .option("header", "true") \
      3     .option("inferSchema", "true") \
      4     .load("abfss://mavenmarket@sgmavenmarkets.dfs.core.windows.net/raw/stores/MavenMarket_Stores.csv")
      6 delta_table = "products_delta_table"
----> 7 raw_df.write.format("delta").mode("overwrite").saveAsTable(delta_table)
      9 display(spark.read.table(delta_table))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observat

In [0]:

new_data = [("3", "Tablet", 500), ("4", "Monitor", 300)]
append_df = spark.createDataFrame(new_data, ["id", "product", "price"])


for i in range(5):
    append_df.write.format("delta").mode("append").saveAsTable(delta_table)

print("Data appended 5 times.")

In [0]:

spark.sql(f"OPTIMIZE {delta_table}")
display(spark.sql(f"DESCRIBE HISTORY {delta_table}"))